In [1]:
import numpy as np
import pandas as pd

In [2]:
import yfinance as yf

df = yf.download('NVDA', start="2010-01-01")
df

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,4.627500,4.655000,4.527500,4.622500,4.240231,80020400
2010-01-05,4.605000,4.740000,4.605000,4.690000,4.302148,72864800
2010-01-06,4.687500,4.730000,4.642500,4.720000,4.329666,64916800
2010-01-07,4.695000,4.715000,4.592500,4.627500,4.244813,54779200
2010-01-08,4.590000,4.670000,4.562500,4.637500,4.253987,47816800
...,...,...,...,...,...,...
2024-04-05,868.659973,884.809998,859.260010,880.080017,880.080017,39885700
2024-04-08,887.000000,888.299988,867.320007,871.330017,871.330017,28322000
2024-04-09,874.419983,876.349976,830.219971,853.539978,853.539978,50354700


In [15]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_df = scaler.fit_transform(df[['Close']].values)

In [16]:
scaled_df

array([[0.00253482],
       [0.00260604],
       [0.00263769],
       ...,
       [0.89820633],
       [0.91598438],
       [0.94417994]])

In [5]:
# Define parameters
n_hidden_units = 128
input_length = 100
output_length = 20
dropout_rate = 0.2
n_features = 1

In [6]:
# Create time series batches
X, Y = [], []
for i in range(len(scaled_df) - input_length - output_length + 1):
    X.append(scaled_df[i:i+input_length])
    Y.append(scaled_df[i+input_length:i+input_length+output_length])

X = np.array(X)
Y = np.array(Y)

In [ ]:
from keras.models import Sequential
from keras.layers import GRU, Dropout, Dense


# Define GRU model
model = Sequential()

model.add(GRU(units=n_hidden_units, input_shape=(input_length, n_features), return_sequences=True))
model.add(Dropout(dropout_rate))
model.add(GRU(units=n_hidden_units))
model.add(Dropout(dropout_rate))
model.add(Dense(units=output_length))

model.compile(optimizer='adam', loss='mse')

In [8]:
model.fit(X, Y, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 11s 97ms/step - loss: 6.9932e-04 - val_loss: 0.0033
Epoch 2/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 8s 90ms/step - loss: 8.1780e-05 - val_loss: 0.0019
Epoch 3/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 8s 91ms/step - loss: 6.3743e-05 - val_loss: 0.0018
Epoch 4/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 8s 95ms/step - loss: 6.5280e-05 - val_loss: 0.0017
Epoch 5/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 8s 89ms/step - loss: 6.0590e-05 - val_loss: 0.0020
Epoch 6/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 8s 93ms/step - loss: 4.8572e-05 - val_loss: 0.0019
Epoch 7/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 8s 91ms/step - loss: 5.1193e-05 - val_loss: 0.0020
Epoch 8/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 8s 92ms/step - loss: 4.1471e-05 - val_loss: 0.0018
Epoch 9/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 8s 93ms/step - loss: 4.0056e-05 - val_loss: 0.0018
Epoch 10/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 8s 93ms/step - loss: 3.8079e-05 - val_loss: 0.0030
Epoch 11/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 9s 109ms/step - loss: 4.3201e-05 - val_loss: 0.0015
Epoch 12/50
87/87

In [17]:
# Predictions
def predict_all(model, data, input_length, output_length):
    predictions = []
    for i in range(len(data) - input_length + 1):
        input_data = data[i:i+input_length].reshape(1, input_length, n_features)
        pred = model.predict(input_data).reshape(output_length)
        predictions.append(pred)
    return np.array(predictions)

scaled_preds = predict_all(model, scaled_df, input_length, output_length)
preds = scaler.inverse_transform(scaled_preds)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━

In [20]:
import plotly.graph_objects as go

candles_back = 450
lpadding = 100
rpadding = 100

rlim = rpadding-candles_back if candles_back > rpadding else None

df_display = df.iloc[-lpadding-output_length-candles_back:rlim]

fig = go.Figure(data=[
    go.Candlestick(
        x=df_display.index,
        open=df_display['Open'],
        high=df_display['High'],
        low=df_display['Low'],
        close=df_display['Close']
    )
])

fig.add_trace(go.Scatter(
    x=df_display.index[lpadding:lpadding+output_length],
    y=preds[-output_length-candles_back-1],
    line_shape='hv',
    mode='lines'
))

fig.add_vrect(
    x0=df_display.index[lpadding],
    x1=df_display.index[lpadding+output_length-1],
    annotation_text="FORECAST",
    annotation_position="top left",
    fillcolor="green",
    opacity=0.2,
    line_width=0
)

fig.show()